In [1]:
import lammps
import random
from tqdm import trange
import pymc3 as pm
import arviz as az
import numpy as np
import theano.tensor as tt
from copy import deepcopy
import matplotlib.pyplot as plt

from poscar_to_lammps import poscar_to_lammps
from pymatgen.io.vasp import Poscar

import sys
from contextlib import contextmanager

In [2]:
def type_shell(core_shell):
    shell_nums = []
    x = 0
    for i in core_shell.values():
        x +=1
        if i:
            x+=1
            shell_nums.append(x)
    return shell_nums

In [3]:
@contextmanager
def custom_redirection(fileobj):
    old = sys.stdout
    sys.stdout = fileobj
    try:
        yield fileobj
    finally:
        sys.stdout = old

core_shell = { 'Li': False , 'Ni': False, 'O': True}
shells = type_shell(core_shell)

# 2. Set charges: values either single number (non-core-shell)
# or a sub-dictionary containing { 'core': core_charge, 'shell': shell_charge }
charges = {'Li': +1.0,
           'Ni': +3.0,
           'O': {'core':  +0.960,
                 'shell': -2.960}}

# 3. Load a POSCAR using pymatgen and write lammps file using poscar_to_lammps
file = 'multicoords.lmp'
with open(file, 'w') as struct:
    with custom_redirection(struct):
        poscar = Poscar.from_file('../poscars/POSCAR1')
        poscar_to_lammps(poscar, core_shell, charges )

In [4]:
elements = ['Li', 'Ni', 'Oc', 'O']
pairs = [('Li','O'), ('Ni','O'), ('O','O')]
springs = ['O']

bpp_def = {'Li_O_A'   : 632.1018,
           'Li_O_rho' : 0.2906,
           'Li_O_C'   : 0.0,
           'Ni_O_A'   : 1582.000,
           'Ni_O_rho' : 0.2882,
           'Ni_O_C'   : 0.000,
           'O_O_A'    : 22844.0000,
           'O_O_rho'  : 0.0870,
           'O_O_C'    : 20.7,
           'O_k'      : 65.0,
           'O_r'      : 0.0}

bpp = deepcopy(bpp_def)

In [5]:
def init():
    lmp = lammps.Lammps(units='metal', style = 'full', args=['-log', 'none', '-screen', 'none'])
    lmp.command('read_data multicoords.lmp')
    
    lmp.command('group cores type 1 2 3')
    lmp.command('group shells type 4')
    
    lmp.command('neighbor 2.0 bin')
    lmp.command('comm_modify vel yes')

    lmp.command('kspace_style ewald 1e-6')
    lmp.command('pair_style buck/coul/long/cs 10.0') #ADD THE CS IF USING CORESHELLS NOT IF RIGID ION
    lmp.command('pair_coeff * * 0 1 0')
    
    lmp.command('bond_style harmonic')
    

    return lmp

In [6]:
def set_potentials(lmp):
    for pair in pairs:
        lmp.command('pair_coeff {} {} {} {} {}'.format(dex(pair[0]),
                                                            dex(pair[1]),
                                                            bpp['{}_{}_A'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_rho'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_C'.format(pair[0], pair[1])]))
    for i, spring in enumerate(springs):
        lmp.command('bond_coeff {} {} {}'.format(i+1,
                                                 bpp['{}_k'.format(spring)],
                                                 bpp['{}_r'.format(spring)]))
    

def dex(elin): return elements.index(elin) + 1

In [7]:
lmp = init()
core_mask = [ atype not in shells for atype in lmp.system.types ]
out = np.zeros([sum(core_mask),3])

set_potentials(lmp)

lmp.command('fix 1 cores setforce 0.0 0.0 0.0')
lmp.command('min_style cg')
lmp.command('minimize 1e-25 1e-25 5000 10000')
lmp.command('unfix 1')
lmp.run(0)
out = lmp.system.forces[core_mask]

In [8]:
maxElement = np.amax(out)
maxElement
# np.argmax(out) #index value

1.826754674579115